## Escuela de Ingeniería en Computación, ITCR 

## Ciencia de Datos

## Preprocesamiento de datos categóricos de alta cardinalidad

**Profesora: María Auxiliadora Mora**


### Introducción

El método de codificación de características categóricas con **baja cardinalidad más conocido es One Hot Encoding**. Este método produce vectores ortogonales y equidistantes para cada categoría. Sin embargo, cuando se trata de características de alta cardinalidad, una codificación One Hot presenta varias deficiencias (Mougan, 2021): 

- (a) la dimensión del espacio de entrada aumenta con la cardinalidad de la variable codificada, 
- (b) las características codificadas son dispersas 
- (c) OneHot Encoding no maneja categorías nuevas.

**Otros métodos, más apropiados a alta cardinalidad de las categorías**

**Hashing** es un método que permite trabajar con alta cardinalidad de datos categóricos. El método consiste en utilizar funciones hash para producir un número fijo de características. El resultado es mayor velocidad y menor uso de memoria, a expensas de la capacidad de hacer una transformación inversa para recuperar el datos. Además, pueden darse colisiones si se establece un número pequeño de características de salida.

Otra opción muy comúnmente utilizada es el **agrupamiento (binning)**, la idea clave es que muchos datos siguen el **principio de Pareto o regla del 80-20 (ley de los pocos vitales)** que establece que  la mayoría de los datos se concentrarán en pocos valores. Un ejemplo simple es la nacionalidad. Hay muchas naciones en el mundo, pero para modelar un fenómeno particular no se usan todas, si no que, se elígen las principales (las más comunes) y se coloca a las demás en la categoría Otros. 

Alternativamente, se utiliza también la codificación **Target Encoding** (o codificación media) que funciona como una solución eficaz para superar el problema de la alta cardinalidad. En esta codificación, las características categóricas se reemplazan con el valor de repetición promedio de cada categoría respectiva. Con esta técnica se maneja el problema de alta cardinalidad y se ordenan las categorías permitiendo una fácil extracción de la información y simplificación del modelo. El principal inconveniente de Target Encoding aparece cuando las categorías con pocas muestras (incluso solo una) se reemplazan por valores cercanos al objetivo deseado. Esto sesga el modelo y lo hace propenso a sobreajustarse. 

**Learned Embedding** es un método que proviene del **Deep Learning y el Procesamiento de Lenguaje Natural (NLP)**. El Learned Embedding utiliza vectores densos para realizar la codificación, la idea general es que la distancia entre los vectores densos tendrá significado para el vocabulario codificado. Por ejemplo, la distancia entre "gato" y "perro" será mucho menor que la distancia entre "gato" y "teclado".

El objetivo de este documento es mostrar el uso de los métodos existentes para codificar datos categóricos y presentar un ejemplo de Learned Embedding. 


### Los datos usado en los ejemplos:

En este ejemplo, se utilizará el conjunto de datos denominado "Breast Cancer Data Set" que se ha estudiado ampliamente en el aprendizaje automático desde la década de 1980. Más información en https://archive.ics.uci.edu/ml/datasets/Breast+Cancer

El conjunto de datos es administrado por la Universidad de California en Irvine (University of California Irvine) y clasifica los datos de pacientes con cáncer de mama como recurrente o no recurrente del cáncer. Es un problema de clasificación binaria. Hay 286 ejemplos y nueve variables de entrada.  Una precisión de clasificación razonable en este conjunto de datos está entre el 68% y el 73% (Brownlee, 2020).

**Información de los atributos (casi todos categóricos)**:

1. Class: no-recurrence-events, recurrence-events
2. age: 10-19, 20-29, 30-39, 40-49, 50-59, 60-69, 70-79, 80-89, 90-99.
3. menopause: lt40, ge40, premeno.
4. tumor-size: 0-4, 5-9, 10-14, 15-19, 20-24, 25-29, 30-34, 35-39, 40-44, 45-49, 50-54, 55-59.
5. inv-nodes: 0-2, 3-5, 6-8, 9-11, 12-14, 15-17, 18-20, 21-23, 24-26, 27-29, 30-32, 33-35, 36-39.
6. node-caps: yes, no.
7. deg-malig: 1, 2, 3.
8. breast: left, right.
9. breast-quad: left-up, left-low, right-up, right-low, central.
10. irradiat: yes, no.



In [ ]:
%pip install keras
%pip install tensorflow
%pip install keras.layers
%pip install pydot
%pip install graphviz

In [7]:
# example of learned embedding encoding for a neural network
from numpy import unique
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# keras y tensorflow
import tensorflow as tf
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import concatenate
from keras.utils import plot_model

# torch 
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


# load the dataset
def load_dataset(filename):
    # load the dataset as a pandas DataFrame
    data = read_csv(filename, header=None)
    print("=============== formato de los datos ==================")
    print(data.head(5))
    
    # retrieve numpy array
    dataset = data.values

    print("=============== valores del dataset ==================")
    print(dataset)

    # split into input (X) and output (y) variables
    X = dataset[:, :-1]
    y = dataset[:,-1]
    
    # format all fields as string
    X = X.astype(str)
    
    print("=============== Y resultante ==================")
    print(y)
    
    # reshape target to be a 2d array
    y = y.reshape((len(y), 1))
    return X, y

# prepare input data
def prepare_inputs(X_train, X_test):
    X_train_enc, X_test_enc = list(), list()
    # label encode each column
    for i in range(X_train.shape[1]):
        le = LabelEncoder()
        le.fit(X_train[:, i])
        # encode
        train_enc = le.transform(X_train[:, i])
        test_enc = le.transform(X_test[:, i])
        # store
        X_train_enc.append(train_enc)
        X_test_enc.append(test_enc)
    return X_train_enc, X_test_enc

# prepare target
def prepare_targets(y_train, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    return y_train_enc, y_test_enc

# load the dataset
X, y = load_dataset('./Data/Data/breast-cancer.csv')
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# prepare input data
X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)




=============== formato de los datos ==================
         0          1        2      3      4    5        6           7      8  \
0  '40-49'  'premeno'  '15-19'  '0-2'  'yes'  '3'  'right'   'left_up'   'no'   
1  '50-59'     'ge40'  '15-19'  '0-2'   'no'  '1'  'right'   'central'   'no'   
2  '50-59'     'ge40'  '35-39'  '0-2'   'no'  '2'   'left'  'left_low'   'no'   
3  '40-49'  'premeno'  '35-39'  '0-2'  'yes'  '3'  'right'  'left_low'  'yes'   
4  '40-49'  'premeno'  '30-34'  '3-5'  'yes'  '2'   'left'  'right_up'   'no'   

                        9  
0     'recurrence-events'  
1  'no-recurrence-events'  
2     'recurrence-events'  
3  'no-recurrence-events'  
4     'recurrence-events'  
=============== valores del dataset ==================
[["'40-49'" "'premeno'" "'15-19'" ... "'left_up'" "'no'"
  "'recurrence-events'"]
 ["'50-59'" "'ge40'" "'15-19'" ... "'central'" "'no'"
  "'no-recurrence-events'"]
 ["'50-59'" "'ge40'" "'35-39'" ... "'left_low'" "'no'"
  "'recurrence-

In [8]:
# summarize
print("=============== Cantidad de datos de entrenamiento y pruebas ==================")
print('Train', X_train_enc)
print('Test', X_test.shape, y_test.shape)

print(X_test)

=============== Cantidad de datos de entrenamiento y pruebas ==================
Train [array([3, 1, 3, 1, 2, 2, 3, 3, 3, 1, 2, 2, 2, 2, 4, 2, 1, 4, 1, 4, 4, 3,
       2, 3, 2, 4, 3, 2, 4, 2, 1, 2, 1, 4, 3, 1, 5, 3, 2, 2, 3, 3, 3, 2,
       4, 3, 1, 4, 4, 5, 3, 2, 3, 3, 4, 2, 2, 3, 2, 2, 2, 4, 1, 2, 3, 3,
       2, 2, 3, 3, 4, 2, 2, 3, 2, 1, 2, 1, 4, 2, 3, 1, 3, 3, 3, 4, 2, 4,
       3, 2, 3, 3, 3, 3, 5, 4, 3, 2, 3, 2, 3, 2, 1, 3, 3, 3, 3, 4, 3, 1,
       3, 4, 4, 1, 3, 2, 3, 4, 3, 3, 4, 4, 3, 3, 4, 2, 3, 2, 4, 1, 2, 3,
       2, 1, 2, 1, 4, 4, 4, 3, 3, 2, 1, 2, 2, 3, 2, 3, 2, 3, 3, 3, 4, 1,
       1, 2, 3, 3, 3, 2, 0, 2, 4, 3, 4, 2, 4, 2, 3, 3, 2, 2, 2, 5, 5, 3,
       5, 1, 2, 2, 2, 2, 4, 4, 3, 2, 2, 3, 4, 4, 4]), array([0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 0, 2,
       0, 2, 2, 0, 0, 2, 1, 2, 2, 2, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2,
       0, 0, 2, 0, 0, 0, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0,
       2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 2

In [ ]:
# prepare output data
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)

# make output 3d
y_train_enc = y_train_enc.reshape((len(y_train_enc), 1, 1))
y_test_enc = y_test_enc.reshape((len(y_test_enc), 1, 1))

#print(y_train_enc)

## Demostración de uso de la capa de Embbeding (sin entrenar)


In [ ]:
# Words must be encoded to pass them to the network
x = torch.tensor([[8,6,22,99,98,73,55,6],
                 [43,65,67,32,22,68,6,2],
                 [5,77,9,44,80,22,67,98]])


In [ ]:
# vectorization of words

vocab_size = 100 
# size of the vector representing the words
embedding_dim = 10  
embedding_layer = nn.Embedding(vocab_size, embedding_dim)

# Embedding layer output
out1 = embedding_layer(x)

print(out1)

In [ ]:
# For the sample data
out1 = embedding_layer(torch.tensor(X_train_enc))
print(out1)

## Ejemplo con Keras (API muy simple construida sobre Tensorflow)

Uso de la capa de Embbeding pero luego de entrenamiento.

In [ ]:
# A MultiLayer Perceptron (MLP) neural network is definde with one hidden layer with 10 nodes, 
# and one node in the output layer for making binary classifications.

# prepare each input head
in_layers = list()
em_layers = list()

# for each characteristic
for i in range(len(X_train_enc)):
    # calculate the number of unique inputs
    n_labels = len(unique(X_train_enc[i]))
    # define input layer
    in_layer = Input(shape=(1,))

    # define embedding layer
    # The embedding layer is trained in the network training process.
    em_layer = Embedding(n_labels, 10)(in_layer)
    # store layers
    in_layers.append(in_layer)
    em_layers.append(em_layer)
# concat all embeddings
merge = concatenate(em_layers)
dense = Dense(10, activation='relu', kernel_initializer='he_normal')(merge)
output = Dense(1, activation='sigmoid')(dense)
model = Model(inputs=in_layers, outputs=output)
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(X_train_enc, y_train_enc, epochs=20, batch_size=16, verbose=2)

# evaluate the keras model
_, accuracy = model.evaluate(X_test_enc, y_test_enc, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))

### Referencias y más ejemplos

[1] Sarkar, D. (2018). Categorical Data.  Recuperado de https://towardsdatascience.com/understanding-feature-engineering-part-2-categorical-data-f54324193e63
    
    
[2] Here’s All you Need to Know About Encoding Categorical Data (with Python code). Recuperado de  https://www.analyticsvidhya.com/blog/2020/08/types-of-categorical-data-encoding/

[4] Scikit-learn (2016). Category Encoders.Recuperado de http://contrib.scikit-learn.org/category_encoders/index.html

[5] Dhasade, G (2020). Ways To Handle Categorical Data With Implementation. Recuperado de https://towardsdatascience.com/ways-to-handle-categorical-data-before-train-ml-models-with-implementation-ffc213dc84ec

Ali, M. (2023). Handling Machine Learning Categorical Data with Python Tutorial. Recuperado de https://www.datacamp.com/tutorial/categorical-data

Mougan, C., Masip,D., Nin, J. & Pujol, O. (2021). Quantile Encoder: Tackling High Cardinality Categorical Features in Regression Problems. Recuperado de https://link.springer.com/chapter/10.1007/978-3-030-85529-1_14

Carey, G. (2003). Coding Categorical Variables. Recuperado de http://psych.colorado.edu/~carey/Courses/PSYC5741/handouts/Coding%20Categorical%20Variables%202006-03-03.pdf

Brownlee, J. (2020). 3 Ways to Encode Categorical Variables for Deep Learning. Recuperado de https://machinelearningmastery.com/how-to-prepare-categorical-data-for-deep-learning-in-python/

Serger, C. (2018). An investigation of categorical variable encoding techniques in machine learning: binary versus one-hot and feature hashing. Recuperado de https://www.diva-portal.org/smash/get/diva2:1259073/FULLTEXT01.pdf

Tabular Modeling Deep Dive. Recuperado de https://github.com/fastai/fastbook/blob/master/09_tabular.ipynb